<a href="https://colab.research.google.com/github/khodjasCode/predicted_prices_of_houses_ML/blob/main/predicted_prices_of_houses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JARAYONLARNI TUSHUNISH

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


ANALITIK YONDASHUV

# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy joylashgan qavat
- `max_levels` - ja'mi qavatlar soni
- `price` - uy narxi

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

MA'LUMOTLARNI O'RGANISH


1.   Malumotlarga qo'yilgan talablar
2.   Malumotlarni yig'ish
3.   Malumotlarni talqin qilish



In [ ]:
df.describe()

,rooms,level,max_levels
count,7565.000000,7565.000000,7565.000000
mean,2.625644,3.699273,6.038202
std,1.085201,2.237275,2.613271
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,4.000000
50%,3.000000,3.000000,5.000000
75%,3.000000,5.000000,9.000000
max,10.000000,19.000000,25.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   location    7565 non-null   object
 1   district    7565 non-null   object
 2   rooms       7565 non-null   int64 
 3   size        7565 non-null   object
 4   level       7565 non-null   int64 
 5   max_levels  7565 non-null   int64 
 6   price       7565 non-null   object
dtypes: int64(3), object(4)
memory usage: 413.8+ KB


MA'LUMOTLARNI TAYYORLASH

In [ ]:
if df.isnull().sum().any():
  df = df.dropna()

In [ ]:
df[df.index == 202]

,location,district,rooms,size,level,max_levels,price
202,"город Ташкент, Яккасарайский район, Баходыра",Яккасарайский,3,119,3,9,Договорная


In [ ]:
df.loc[df['size'] == 'Площадьземли:1сот', 'size'] = 100

In [ ]:
df[df.index == 5347]

,location,district,rooms,size,level,max_levels,price
5347,"город Ташкент, Яшнободский район, Дархон",Яшнободский,4,100,3,5,150000


In [ ]:
df['size'] = pd.to_numeric(df['size']).astype(int)

In [ ]:
df['price'] = pd.to_numeric(df['price']).astype(int)

In [ ]:
df.drop(df[df.price == 'Договорная'].index, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7466 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   location    7466 non-null   object
 1   district    7466 non-null   object
 2   rooms       7466 non-null   int64 
 3   size        7466 non-null   int64 
 4   level       7466 non-null   int64 
 5   max_levels  7466 non-null   int64 
 6   price       7466 non-null   object
dtypes: int64(4), object(3)
memory usage: 466.6+ KB


In [ ]:
df.describe()

,rooms,size,level,max_levels,price
count,7466.000000,7466.000000,7466.000000,7466.000000,7.466000e+03
mean,2.622288,113.522368,3.693678,6.023841,7.133421e+04
std,1.083200,1501.057858,2.236770,2.606955,6.405237e+05
min,1.000000,1.000000,1.000000,1.000000,2.000000e+00
25%,2.000000,50.000000,2.000000,4.000000,3.500000e+04
50%,3.000000,65.000000,3.000000,5.000000,4.650000e+04
75%,3.000000,85.000000,5.000000,9.000000,6.700000e+04
max,10.000000,70000.000000,19.000000,25.000000,5.200000e+07


In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

x = train_set.drop('price', axis = 1)
y = train_set['price'].copy()

In [ ]:
cat_feat = ['location', 'district']
num_feat = ['rooms', 'size', 'level', 'max_levels']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
cat_pipeline = Pipeline([('onehot', OneHotEncoder(handle_unknown = 'ignore'))])
num_pipeline = Pipeline([('num', StandardScaler())])

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_feat),
    ('num', num_pipeline, num_feat)
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_prep = preprocessor.fit_transform(x)

In [ ]:
x_prep.toarray()

array([[ 0.        ,  0.        ,  0.        , ..., -0.05730332,
        -1.20598257, -1.54923334],
       [ 0.        ,  0.        ,  0.        , ..., -0.02539645,
        -0.76132443, -0.78536102],
       [ 0.        ,  0.        ,  0.        , ..., -0.04879482,
        -1.20598257, -0.78536102],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.10152203,
        -1.20598257,  0.74238361],
       [ 0.        ,  0.        ,  0.        , ..., -0.04524962,
         0.12799185, -0.40342486],
       [ 0.        ,  0.        ,  0.        , ..., -0.03603207,
         0.12799185, -0.78536102]])

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [ ]:
LR_model.fit(x_prep, y)

LinearRegression()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(x_prep, y)

RandomForestRegressor()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [ ]:
lr_predictions = LR_model.predict(x_prep)
lr_mae = mean_absolute_error(y, lr_predictions)
lr_rmse = np.sqrt(mean_squared_error(y, lr_predictions))
print(f"Linear Regression - MAE: {lr_mae:.2f}, RMSE: {lr_rmse:.2f}")

Linear Regression - MAE: 18319.12, RMSE: 176226.61


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
rf_predictions = RF_model.predict(x_prep)
rf_mae = mean_absolute_error(y, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y, rf_predictions))
print(f"Random Forest - MAE: {rf_mae:.2f}, RMSE: {rf_rmse:.2f}")

Random Forest - MAE: 6815.06, RMSE: 91899.16


In [ ]:
import joblib

In [ ]:
joblib.dump(LR_model, 'LR_model_uy.pkl')

['LR_model_uy.pkl']

In [ ]:
joblib.dump(RF_model, 'RF_model_uy.pkl')

['RF_model_uy.pkl']

In [ ]:
custom_data = pd.DataFrame({
    'location': ['город Ташкент, Юнусабадский район, Юнусабад 8-й квартал'],
    'district': ['Юнусабадский'],
    'rooms': [3],
    'size': [70],
    'level': [5],
    'max_levels': [9]
})

In [ ]:
custom_data

,location,district,rooms,size,level,max_levels
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,70,5,9


In [ ]:
custom_data_prepared = preprocessor.transform(custom_data)

In [ ]:
predicted_price = RF_model.predict(custom_data_prepared)
print(f"Predicted price for the custom input: {predicted_price[0]:.2f}")

Predicted price for the custom input: 53849.83


In [ ]:
predicted_price = LR_model.predict(custom_data_prepared)
print(f"Predicted price for the custom input: {predicted_price[0]:.2f}")

Predicted price for the custom input: 65149.40


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
